# Train TextCNN
Stefan/Yuzhao Heng
Since Wed. Mar. 16th. 2022


## Setup
### Ipython



In [1]:
%load_ext autoreload
%autoreload 2


### Colab


In [2]:
import os
import sys


is_on_colab = 'google.colab' in sys.modules

if is_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream
    ! pip3 install wfdb
    ! pip3 install pytorch-lightning transformers datasets
    ! pip3 install vit-pytorch

    path = os.path.join('drive', 'My Drive', 'Research', 'ECG Classification', 'ECG-Representation-Learning')
    sys.path.append(path)
    ! ls "{path}"

    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    import torch
    if torch.cuda.is_available():
        ! nivida-smi


from icecream import ic

from ecg_transformer.util import *
ic(PATH_BASE, DIR_PROJ, PKG_NM)



/Users/stefanh/Documents/UMich/Course/Course 2022 Winter/EECS487
Text-CNN-SQuAD
text_cnn


### code


In [3]:
from pytorch_lightning.utilities.seed import seed_everything
from icecream import ic

from ecg_transformer.baseline import MyTrainer

seed_everything(config('random-seed'))



## Prep

In [4]:
trainer = MyTrainer(train_args=dict(
    train_batch_size=4,
    eval_batch_size=4,
    n_sample=8,
    precision=16
))



Loaded KeyedVectors from disk at /Users/stefanh/Documents/UMich/Course/Course 2022 Winter/EECS487/Text-CNN-SQuAD/glove-wiki-gigaword-300.kv


ic| md: TextCnn(
          (embed_frozen): Embedding(400000, 300)
          (embed_train): Embedding(5, 300)
          (embed_position): Embedding(88, 300)
          (conv_module): OriginalConvModule(
            (convs): ModuleList(
              (0): Conv2d(1, 64, kernel_size=(3, 300), stride=(1, 1))
              (1): Conv2d(1, 64, kernel_size=(4, 300), stride=(1, 1))
              (2): Conv2d(1, 64, kernel_size=(5, 300), stride=(1, 1))
            )
          )
          (dropout): Dropout(p=0.5, inplace=False)
          (fc): Linear(in_features=192, out_features=2, bias=True)
          (loss_fn): CrossEntropyLoss()
        )


## Train



In [5]:
trainer.train()



Loaded KeyedVectors from disk at /Users/stefanh/Documents/UMich/Course/Course 2022 Winter/EECS487/Text-CNN-SQuAD/glove-wiki-gigaword-300.kv
2022-03-17 20:45:47| [TextCNN Train]::train::train.py:135, INFO - Launched training model {model name: TextCnn, pretrained embedding shape: (400000, 300), trainable embedding shape: (5, 300), max sentence length: 64, max question length: 24, conv module meta: {name=original, kernel-sp=(3, 4, 5) x 64, pool=avg}, #class: 2, loss weight: None} with args {#train: 4, #train pair: 2, #eval: 4, train batch size: 64, eval batch size: 32, #epoch: 4, #step: 8, learning rate: 0.0003, warmup ratio: 0.05, negative sub-sampling: True} and save logging False... 
2022-03-17 20:45:47| [TextCNN Train]::evaluate::train.py:206, INFO - Running evaluation with args {#eval group: 4, eval batch size: 32} at epoch -1


Eval Before training: 100%|██████████| 1/1 [00:00<00:00,  3.06ba/s]


2022-03-17 20:45:47| [TextCNN Train]::evaluate::train.py:283, INFO - {eval_loss:  0.7145, eval_binary_acc:   9.52, eval_cls_acc:   7.50}


Train Epoch #1/4: 100%|██████████| 2/2 [00:00<00:00,  5.19it/s]


2022-03-17 20:45:48| [TextCNN Train]::train::train.py:187, INFO - Last batch: {epoch: 0/4, step: 2/8, lr: 2.561e-04, train_loss:  0.7994, train_binary_acc:  37.50}
2022-03-17 20:45:48| [TextCNN Train]::evaluate::train.py:206, INFO - Running evaluation with args {#eval group: 4, eval batch size: 32} at epoch 0


...

2022-03-17 20:45:49| [TextCNN Train]::evaluate::train.py:283, INFO - {eval_loss:  0.6474, eval_binary_acc:  76.19, eval_cls_acc:  25.00}


Train Epoch #4/4: 100%|██████████| 2/2 [00:00<00:00,  6.62it/s]


2022-03-17 20:45:49| [TextCNN Train]::train::train.py:187, INFO - Last batch: {epoch: 3/4, step: 8/8, lr: 0.000e+00, train_loss:  0.6924, train_binary_acc:  62.50}
2022-03-17 20:45:49| [TextCNN Train]::evaluate::train.py:206, INFO - Running evaluation with args {#eval group: 4, eval batch size: 32} at epoch 3


Eval Epoch #4/4: 100%|██████████| 1/1 [00:00<00:00,  4.20ba/s]


2022-03-17 20:45:50| [TextCNN Train]::evaluate::train.py:283, INFO - {eval_loss:  0.6740, eval_binary_acc:  66.67, eval_cls_acc:  10.00}


## Evaluate



In [6]:
# TODO



2022-03-17 20:45:50| [TextCNN Eval]::evaluate::train.py:213, INFO - Launched evaluation on model {model name: TextCnn, pretrained embedding shape: (400000, 300), trainable embedding shape: (5, 300), max sentence length: 64, max question length: 24, conv module meta: {name=original, kernel-sp=(3, 4, 5) x 64, pool=avg}, #class: 2, loss weight: None} with args {#eval group: 4, eval batch size: 32}


100%|██████████| 1/1 [00:00<00:00,  4.42ba/s]


2022-03-17 20:45:50| [TextCNN Eval]::evaluate::train.py:283, INFO - {eval_loss:  0.6740, eval_binary_acc:  66.67, eval_cls_acc:  10.00}


{'eval_loss': ' 0.6740', 'eval_binary_acc': ' 66.67', 'eval_cls_acc': ' 10.00'}